In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from datetime import date
from os.path import join
from nltk.stem.snowball import SnowballStemmer
from sklearn.pipeline import Pipeline
import csv

In [2]:
from parseUtils import parseDocs, update_file
out_path = ("/Volumes/Media/Documents/Git/MachineLearning/src"
            "/main/resources/classifierOuts/")

tagDict, tagList, textList, namesList = parseDocs()

In [3]:
with open('stop_words.txt') as sw:
    stop_words = [line.rstrip('\n') for line in sw]

with open('important_words.txt') as iw:
    imp_words = {line.rstrip('\n') for line in iw}

print(len(tagDict))

36


In [4]:
trainNum = 3684
testNumStart = 3685
testNumFinish = len(textList)

In [5]:
##############
p = '/Volumes/Media/Documents/Git/MachineLearning/out/validation/'
__, __, texts_to_pred, names = parseDocs(tag = 'notag',
                                         texts_used = namesList,
                                         discard_tags = False,
                                         mypath = p)

textList.extend(texts_to_pred)
##############

In [6]:
stemmer = SnowballStemmer('russian')
tkzr = CountVectorizer().build_tokenizer()

stem_tokenize = lambda tokens: [stemmer.stem(item) for item in tokens]
tokenize = lambda text: stem_tokenize(tkzr(text))


prepoc = Pipeline([('count', CountVectorizer(tokenizer=tokenize,
                                             max_df=0.8,
                                             min_df=0.01,
                                             ngram_range=(1, 2),
                                             stop_words = stop_words)),
                   ('tfidf', TfidfTransformer())])

In [7]:
# Creating multilaber tags by binarizing them

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(tagList)
x = prepoc.fit_transform(textList)
x.shape

(6156, 8905)

In [8]:
clf = Pipeline([('chi2', SelectKBest(chi2, k = 'all')),
                ('svm', LinearSVC(random_state=0,
                                  class_weight='balanced',
                                  C=2.0))])

In [9]:
y.shape

(4606, 36)

In [10]:
classer = OneVsRestClassifier(clf, n_jobs=1).fit(x[:trainNum, :],
                                                y[:trainNum, :])

/Volumes/Media/Applications/Anaconda/anaconda/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:165: RuntimeWarning: invalid value encountered in true_divide
  chisq /= f_exp
/Volumes/Media/Applications/Anaconda/anaconda/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:165: RuntimeWarning: invalid value encountered in true_divide
  chisq /= f_exp
/Volumes/Media/Applications/Anaconda/anaconda/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:165: RuntimeWarning: invalid value encountered in true_divide
  chisq /= f_exp
/Volumes/Media/Applications/Anaconda/anaconda/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:165: RuntimeWarning: invalid value encountered in true_divide
  chisq /= f_exp
/Volumes/Media/Applications/Anaconda/anaconda/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:165: RuntimeWarning: invalid value encountered in true_divide
  

In [11]:
import evaluation as e

In [12]:
from imp import reload
reload(e)

<module 'evaluation' from '/Volumes/Media/Documents/Git/MachineLearning/src/main/python/classifier/evaluation.py'>

In [13]:
ev = e.eval(out_path, classer, mlb, x, tagList, namesList)
ev.evalTrainer(testNumStart, testNumFinish)

macro_precision = 0.6590662323561346
micro_precision = 0.8276595744680851
recall = 0.7649950835791544
F1 = 0.795094532447624
macro_correct = 607
micro_correct = 778
false_neg_tags = 239
false_pos_tags = 162
total_test_docs = 921
n_of_pred_tags = 940
n_of_relevant_tags = 1017


In [14]:
x.shape

(6156, 8905)

In [15]:
pred = classer.predict(x[testNumFinish + 1 : x.shape[0] - 1, : ])

In [16]:
import parseUtils as pu
reload(pu)
path = '/Volumes/Media/Documents/Git/MachineLearning/out/validation/'
for i in range(0, pred.shape[0]):
    tags_pred = []
    predicted = pred[i]
    for j in predicted.nonzero()[0]:
        tags_pred.append(mlb.classes_.item(j))
    if tags_pred:
        pu.update_file(path, names[i], tags_pred)